In [1]:
#Ensemble de arboles de decision
#utilizando el naif metodo de Arboles Azarosos
#entreno cada arbol utilizando un subconjunto distinto de atributos del dataset

#limpio la memoria
rm( list=ls() )  #Borro todos los objetos
gc()   #Garbage Collection

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,614668,32.9,1321371,70.6,1321371,70.6
Vcells,1093283,8.4,8388608,64.0,1632151,12.5


In [2]:
require("data.table")
require("rpart")

Loading required package: data.table

Warning message:
"package 'data.table' was built under R version 4.2.3"
Loading required package: rpart

Warning message:
"package 'rpart' was built under R version 4.2.3"


In [4]:
#parmatros experimento
PARAM <- list()
PARAM$experimento  <- 7.3210
PARAM$semilla  <- 604997      #Establezco la semilla aleatoria, cambiar por SU primer semilla

In [5]:
#parameetros rpart
PARAM$rpart_param   <- list( "cp"=          -1,
                              "minsplit"=  2000,
                              "minbucket"=  100,
                              "maxdepth"=   10 )

In [6]:
#parametros  arbol
PARAM$feature_fraction  <- 0.5  #entreno cada arbol con solo 50% de las variables variables
PARAM$num_trees_max  <- 500 #voy a generar 500 arboles, a mas arboles mas tiempo de proceso y MEJOR MODELO, pero ganancias marginales

In [7]:
#------------------------------------------------------------------------------
#------------------------------------------------------------------------------
#Aqui comienza el programa

setwd("C:/Users/ccarl/Documents/labo2023v/src")  #Establezco el Working Directory

In [8]:
#cargo los datos
dataset  <- fread("./datasets/dataset_pequeno.csv")

In [9]:
#creo la carpeta donde va el experimento
dir.create( "./exp/", showWarnings = FALSE  )
carpeta_experimento  <-  paste0( "./exp/7Lab", PARAM$experimento, "/")
dir.create( paste0( "./exp/7Lab", PARAM$experimento, "/"), 
            showWarnings = FALSE )

setwd( carpeta_experimento )

In [10]:
#que tamanos de ensemble grabo a disco, pero siempre debo generar los 500
grabar  <-  c( 1, 5, 10, 50, 100, 200, 500)

In [11]:
#defino los dataset de entrenamiento y aplicacion
dtrain  <- dataset[ foto_mes==202107 ]
dapply  <- dataset[ foto_mes==202109 ]

In [12]:
#aqui se va acumulando la probabilidad del ensemble
dapply[ , prob_acumulada := 0 ]


In [13]:
#Establezco cuales son los campos que puedo usar para la prediccion
#el copy() es por la Lazy Evaluation
campos_buenos  <- copy( setdiff(  colnames(dtrain) ,  c("clase_ternaria") ) )

In [14]:
#Genero las salidas
set.seed(PARAM$semilla) #Establezco la semilla aleatoria

In [15]:
for( arbolito in  1:PARAM$num_trees_max )
{
  qty_campos_a_utilizar  <- as.integer( length(campos_buenos)* PARAM$feature_fraction )
  campos_random  <- sample( campos_buenos, qty_campos_a_utilizar )
  
  #paso de un vector a un string con los elementos separados por un signo de "+"
  #este hace falta para la formula
  campos_random  <- paste( campos_random, collapse=" + ")

  #armo la formula para rpart
  formulita  <- paste0( "clase_ternaria ~ ", campos_random )

  #genero el arbol de decision
  modelo  <- rpart( formulita,
                    data= dtrain,
                    xval= 0,
                    control= PARAM$rpart_param )

  #aplico el modelo a los datos que no tienen clase
  prediccion  <- predict( modelo, dapply , type = "prob")
  
  dapply[  ,  prob_acumulada :=  prob_acumulada + prediccion[ , "BAJA+2"] ]

  if( arbolito %in%  grabar )
  {
    #Genero la entrega para Kaggle
	umbral_corte  <-  (1/40) *  arbolito
    entrega  <- as.data.table( list( "numero_de_cliente"= dapply[  , numero_de_cliente],
                                     "Predicted"= as.numeric(dapply[  , prob_acumulada] > umbral_corte) ) ) #genero la salida

    nom_arch  <- paste0('KA', PARAM$experimento, "_",
						sprintf( '%.3d', arbolito ), #para que tenga ceros adelante
						'.csv' )
    fwrite( entrega, 
            file= nom_arch,
            sep= "," )

    cat( arbolito, " " )
  }
  
}


1  5  10  50  100  200  500  